# Hierarchical Embeddings
Taking our already generated embeddings form Cohere's Embed V3 Model, and training these embeddings in a hierarchical fashion. This will be achieved by sparse encoding the embeddings and integrating findings from the Matryoshka Embeddings Model. 

Sparse encoding will generate progressively simplified embeddings by reducing the dimensionality of the data by setting elements to zero based on a loss term. This means that we will be capturing the most important features in the lower dimensions (this is similar to the Matryoshka Idea), which mimics hierarchical embeddings. 

In most embeddings models, typically a loss function is applied on the full-size embedding to ensure the quality of the created embedding. However, in the Matryoshka framework, not only is the loss function applied to the full-size embeddings, but to dimensionally reduced (truncated) portions of the embeddings as well.

"Shortlisting and reranking: Rather than performing your downstream task (e.g., nearest neighbor search) on the full embeddings, you can shrink the embeddings to a smaller size and very efficiently "shortlist" your embeddings. Afterwards, you can process the remaining embeddings using their full dimensionality."

In [1]:
#imports
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, backend as K

### Preprocessing the Embeddings
This may not be necessary since we will be using Cohere's Embed V3 model

In [2]:
#use cohere embed v3 - here will use a place holder for testing purposes

#define the embeddings
embeddings = np.random.rand(1000, 256) #example

#instantiate the StandardScaler class
scaler = StandardScaler()

#fit the scaler to the embeddings
scaled_embeddings = scaler.fit_transform(embeddings)

### Defining a Sparse Autoencoder with Matryoshka Loss
Following the Matryoshka framework, we will train a sparse autoencoder for different embedding dimensions. The goal is that the smallest dimensions perserve the most important data/features. Nested loss will be applied to penalize the lower-dimensional embeddings for losing important information.

Sparse autoencoders create a compressed representation, enforcing sparsity via a loss term. Autoencoders consist of an encoder, which compresses the input into a lower-dimensional embedding; the decoser reconstructs the input from the compressed versions.

In the encoding layer, an L1 regularization term is added as the sparsity constraint (it enforces sparsity on the embeddings (known as sparse codes)). L1 regularizationn works by encouraging many of the weights to be zero, which creates sparsity in the latent space (the 128-dim encoded space). As a result, this ensures that only a few dimensions carry signifiacnt informtion.  



In [4]:
#defining the input dimension
input_dim = scaled_embeddings.shape[1]

### defining the layers of the autoencoder ###

#input_dim is 256; size of the original embedding
input_layer = Input(shape = (input_dim,)) 

#compresses the input into a lower-dimensional representation; reduces dim to 128
#regularizers.l1(1e-5) adds a sparsity constraint by penalizing large numbers of non-zero values in the embedding
encoded = Dense(128, activation='relu', activity_regularizer=regularizers.l1(1e-5))(input_layer) 

 #reconstructs dim back to 256
decoded = Dense(input_dim, activation='sigmoid')(encoded)

#building the autoencoder
autoencoder = Model(input_layer, decoded)

The loss function will be modified so that it works across multiple embedding sizes. Here, we will use the mean squared error (MSE) for each dimension. The loss ensures that the smaller-scaled embeddings capture the most important/essential info. 

In [5]:
#defining the Matryoshka loss
#assuming for this example that we are looking at the following dimensions: 64, 128, 256
# ensures that lower-dimensional embeddings retain the most important info

def matryoshka_loss(y_true, y_pred):
    # Loss for 64-dimensional embedding
    mse_64 = K.mean(K.square(y_true[:, :64] - y_pred[:, :64]))

    # Loss for 128-dimensional embedding
    mse_128 = K.mean(K.square(y_true[:, :128] - y_pred[:, :128]))

    # Loss for 264-dimensional embedding
    mse_256 = K.mean(K.square(y_true - y_pred))

    # Combine losses with increasing weight for larger dimensions
    #combine losses with different weights to ensure that smaller dimensions are prioritized
    return mse_64 + mse_128 * 0.5 + mse_256 * 0.25



In [7]:
#compiling the autoencoder with the customized loss function, and using Adam as the optimizer
autoencoder.compile(optimizer='adam', loss=matryoshka_loss)

#building the encoder to extract sparse codes
encoder = Model(input_layer, encoded)

### Training the Autoencoder
Training the autoencoder on Cohere's Embed v3 model. The sparsity constraint forces the model to focus on the most important features. The autoencoder is trained by minimizing the Matryoshka loss using the Adam optimizer. We train it in an unsupervised fashion by using the generated embeddings as the input and the output. 

In [8]:
autoencoder.fit(scaled_embeddings, scaled_embeddings,
                epochs=100, #100 training cycles
                batch_size=256,#256 embeddings per training step
                shuffle=True, # randomized the data each epoch for better generalization
                validation_split=0.2)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 2.3704 - val_loss: 2.2643
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.2684 - val_loss: 2.1803
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.1767 - val_loss: 2.1099
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.1056 - val_loss: 2.0524
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.0495 - val_loss: 2.0067
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.9923 - val_loss: 1.9714
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.9639 - val_loss: 1.9444
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.9354 - val_loss: 1.9241
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.9187 - val_loss: 1.9087
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.9018 - val_loss: 1.8970
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.8849 - val_loss: 1.8880
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.8790 - val_lo

### Extracting Sparse Codes
Once the autoencoder is trained, we will use the trained encoder part of the model to extract sparse codes from the embeddings, which are the compressed 128-dimensional representation. 

In [10]:
#extracting sparse codes from the scaled embeddings 
sparse_codes = encoder.predict(scaled_embeddings)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


### Hierarchical Decomposition
We now have the sparse codes in 128 dimensions from the previous step. To create the hierarchical representations (aka the embeddings of different sizes), progressively truncate the sparse codes by removing the smallest components by setting them to zero. As it stands, the most important part of the data (most important features) are in the largest components. For each embedding vector, this function finds the top k largest absolute values and keeps them, setting the rest to zero.

In [11]:
def truncate_sparse_codes(sparse_codes, k):
    truncated_codes = np.copy(sparse_codes)
    for i in range(truncated_codes.shape[0]):
        threshold = np.sort(np.abs(truncated_codes[i]))[-k]
        truncated_codes[i][np.abs(truncated_codes[i]) < threshold] = 0
    return truncated_codes


#example outputs 
#generating hierarchical embeddings at different scales
# Generate hierarchical embeddings at different scales
sparse_codes_64 = truncate_sparse_codes(sparse_codes, 64)
sparse_codes_128 = truncate_sparse_codes(sparse_codes, 128)

In [15]:
print("Sparse Codes dim = 64: ", sparse_codes_64)
print("Sparse Codes dim = 128: ", sparse_codes_128)

Sparse Codes dim = 64:  [[0.         0.         0.08174219 ... 0.         0.         0.        ]
 [0.00709551 0.         0.         ... 0.         0.39538938 1.5206007 ]
 [1.6163566  0.         0.2362078  ... 0.80892277 0.         2.8570728 ]
 ...
 [0.         1.3171427  1.0912931  ... 1.3575659  0.         2.1430118 ]
 [0.         1.4688016  0.97833616 ... 0.7575442  0.         2.5488367 ]
 [1.8490803  0.         0.         ... 1.2552141  0.         0.        ]]
Sparse Codes dim = 128:  [[0.         0.         0.08174219 ... 0.         0.         0.        ]
 [0.00709551 0.         0.         ... 0.         0.39538938 1.5206007 ]
 [1.6163566  0.         0.2362078  ... 0.80892277 0.         2.8570728 ]
 ...
 [0.         1.3171427  1.0912931  ... 1.3575659  0.         2.1430118 ]
 [0.         1.4688016  0.97833616 ... 0.7575442  0.         2.5488367 ]
 [1.8490803  0.         0.         ... 1.2552141  0.         0.        ]]


### Evaluation
Can perform several tasks to evaluate the quality of the hierarchical embeddings. Such examples include: performing classification tasks, and comparing the F1/accuracy scores of the embeddings of different dimensions; applying clustering algorithms on the embeddings and evaluate the qulaity using metrics like Silhouette Score or ARI; evaluating how well the different dimensional embeddings capture semantic similarity by comparing them using cosine similarity between different embeddings for similar data points.

In [16]:
#Similarity Search using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

# Compare cosine similarity between embeddings
similarity_64 = cosine_similarity([sparse_codes_64[0]], sparse_codes_64[1:])
similarity_128 = cosine_similarity([sparse_codes_128[0]], sparse_codes_128[1:])

In [17]:
print(similarity_64)
print(similarity_128)

[[0.35277212 0.25457639 0.24723174 0.22666414 0.16883938 0.18549293
  0.28276008 0.28534466 0.23341233 0.10270618 0.33972818 0.21255778
  0.26833817 0.11859377 0.21312326 0.17165896 0.18962568 0.23877868
  0.23207588 0.17904395 0.13681963 0.27828786 0.18154493 0.23019323
  0.14465822 0.2112618  0.19978246 0.31044215 0.14063604 0.2607712
  0.09304354 0.3707789  0.209468   0.18064101 0.20050962 0.28076887
  0.25279182 0.16159503 0.25720677 0.28996044 0.351972   0.12095335
  0.28176618 0.2940996  0.2968707  0.21730533 0.20844576 0.1609982
  0.20721133 0.2046988  0.19591802 0.18383698 0.1703535  0.18491301
  0.12382811 0.40328285 0.17568259 0.20857623 0.27838486 0.26923504
  0.13365757 0.26844904 0.21613674 0.36233747 0.13077179 0.15937048
  0.10936852 0.4085396  0.26471883 0.3102298  0.228241   0.31223637
  0.19189686 0.14549994 0.14821061 0.13586614 0.1539326  0.3207196
  0.19800557 0.22057378 0.28113267 0.11177672 0.31589055 0.23438735
  0.20107937 0.2952747  0.38731068 0.27042094 0.141